Fitness fucntion is the number of the rows, ouputed correctly by the chromosom. It also takes the negative of the circuits into the account as well.
I defined a cutoff probability. The chromoses who are in the bottom of the given percentile, will not be selected for the replication
Crossover chance will decrease gradually as the generations pass. The lowest value of the crossover would be 0.15 the mutation chance is quite low though: 0.05 ideally.
Sometimes it is kilely tha GA covnerges to local minima. The reason for that is the survived chromosomes cannot yield better solution by getting crossovered or mutated. This leads to slow convergence or not  converging to the final solution at all. 
One solution is to make the chance of mutation higher and thus increase the randomness of the algorithm which leads in escaping the local minima.

In [12]:
import pandas as pd
import numpy as np
import copy 
import random

In [13]:
class And_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'and'

  def khorooj(self):
    if self.in1 ==1 and self.in2 == 1:
      self.khorooji = 1
      return 1 

    self.khorooji = 0
    return 0 



In [14]:
class Or_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'or'
  
  def khorooj(self):
    if self.in1 ==1 or self.in2 == 1:
      self.khorooji = 1
      return 1 

    self.khorooji = 0 
    return 0 


In [15]:
class Xor_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'xor'
  
  def khorooj(self):
    if (self.in1 ==1 and self.in2 == 0) or (self.in1 ==0 and self.in2 == 1):
      self.khorooji = 1
      return 1 

    self.khorooji = 0
    return 0 

In [16]:
class Nand_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'nand'
  
  def khorooj(self):
    if self.in1 ==1 and self.in2 == 1:
      self.khorooji = 0
      return 0

    self.khorooji = 1
    return 1 

In [17]:
class Nor_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'nor'
  
  def khorooj(self):
    if self.in1 ==1 or self.in2 == 1:
      self.khorooji = 0
      return 0

    self.khorooji = 1
    return 1 

In [18]:
class Xnor_gate:
  def __init__(self, in1=0, in2=0):
    self.in1 = in1 
    self.in2 = in2 
    self.khorooji = 0
    self.name = 'xnor'
  
  def khorooj(self):
    if (self.in1 ==1 and self.in2 == 0) or (self.in1 ==0 and self.in2 == 1):
      self.khorooji = 0
      return 0

    self.khorooji = 1
    return 1

In [19]:
class Chromosom:
  def __init__(self, genes):
    self.genes = genes
    self.khorooji = 0
    self.fitness = 0
    

  def get_names(self):
    names = []
    for g in self.genes:
      names.append(g.name)
    
    return names 

  def khorooj(self, inputs):
    self.genes[0].in1 = inputs[0]
    for cnt in range(9):
      self.genes[cnt].in2 = inputs[cnt + 1]
      if cnt > 0:
        self.genes[cnt].in1 = self.genes[cnt-1].khorooj()
      

    self.khorooji = self.genes[-1].khorooj()
    return self.khorooji 

In [28]:
class Fitness:
  def __init__(self, tt_add, in_num):
    self.tt_df = pd.read_csv(tt_add)
    self.tt_np  = self.tt_df.to_numpy(dtype=np.int8)
    self.in_num = in_num 


  def check_the_sol(self, popu):
    for p in popu:
      if p.fitness == 2**self.in_num:
        return p 

    return None 
    
  def check_fitness(self, chro):
    fit_cnt = 0
    for row in self.tt_np:
      khorooji = chro.khorooj(row[:-1])
      if khorooji == row[-1]:
        fit_cnt += 1
      
      
    chro.fitness = fit_cnt 
    return max(fit_cnt, 2**self.in_num - fit_cnt)


  def check_real_fitness(self, chro):
    fit_cnt = 0
    for row in self.tt_np:
      khorooji = chro.khorooj(row[:-1])
      if khorooji == row[-1]:
        fit_cnt += 1
      
      
    chro.fitness = fit_cnt 
    return fit_cnt


In [21]:
class Genetic:
  def __init__(self, fit, init_pop, mut_chance, cross_chance, die_cutoff):
    self.init_pop = init_pop 
    self.mut_chance = mut_chance
    self.die_cutoff = die_cutoff
    self.cross_chance = cross_chance 
    self.preserve_cross = cross_chance 
    self.fit = fit 
    self.best = Chromosom(None) 
    


  def create_random_chro(self, length):
    rnds = np.random.randint(0, 6, length)
    genes = []
    for rnd in rnds:
      if rnd == 0:
        genes.append(And_gate())
      elif rnd == 1:
        genes.append(Or_gate())
      elif rnd == 2:
        genes.append(Xor_gate())
      elif rnd == 3:
        genes.append(Nand_gate())
      elif rnd == 4:
        genes.append(Nor_gate())
      elif rnd == 5:
        genes.append(Xnor_gate())
    
    return Chromosom(genes)


  def create_random_pop(self):
    popu = [self.create_random_chro(9) for _ in range(self.init_pop)]
    return popu 

  def crossover(self, chr1, chr2):
    point = np.random.randint(1, 8)

    for cnt in range(8, 8-point, -1):
      tmp = copy.deepcopy(chr1.genes[cnt])
      chr1.genes[cnt] = copy.deepcopy(chr2.genes[cnt])
      chr2.genes[cnt] = tmp 

    chr1.fitness = 0
    chr2.fitness = 0

    return chr1, chr2   

  def mutate(self, chro):
    gate = np.random.randint(0, 6)
    point = np.random.randint(0, 9)

    if gate == 0:
      chro.genes[point] = And_gate()
    elif gate == 1:
      chro.genes[point] = Or_gate()
    elif gate == 2:
      chro.genes[point] = Xor_gate()
    elif gate == 3:
      chro.genes[point] = Nand_gate()
    elif gate == 4:
      chro.genes[point] = Nor_gate()
    elif gate == 5:
      chro.genes[point] = Xnor_gate()

    chro.fitness = 0

    return chro

  
  def kill_the_pop(self, popu):

    
    
    popu = sorted(popu, key=lambda x: x.fitness)
    print(popu[-1].fitness)

    if popu[-1].fitness > self.best.fitness:
      self.best = copy.deepcopy(popu[-1])

    
    last_len = len(popu)
    
    popu = popu[int(len(popu)*self.die_cutoff):len(popu)]

    if len(popu) == last_len:
      return None 

    return popu 


  def create_new_pop(self, popu):
    mu_list = []
    cross_list = []
    norm_list = []
    random.shuffle(popu)
    for p in popu:
      mu = True if np.random.rand() < self.mut_chance else False 
      cr = True if np.random.rand() < self.cross_chance else False 
      self.cross_chance = float(len(popu) / self.init_pop) * (self.preserve_cross-0.15) + 0.15 
      if mu == True:
        mu_list.append(p)

      elif cr == True:
        cross_list.append(p)
      
      else:
        norm_list.append(p)

      
    if len(cross_list) % 2 != 0:
      cross_list.append(copy.deepcopy(cross_list[0]))
    
    for ind, m in enumerate(mu_list):
      mu_list[ind] = self.mutate(m)

    for cnt in range(0, len(cross_list)-1, 2):
      chr1, chr2 = self.crossover(cross_list[cnt], cross_list[cnt+1])
      cross_list[cnt], cross_list[cnt+1] = chr1, chr2

    return cross_list + mu_list + norm_list 

    

    

  def run_gen(self):

    popu = self.create_random_pop()
    while True:
      mian = 0
      for ind, p in enumerate(popu):
        if p.fitness == 0:
          p.fitness = self.fit.check_fitness(p)

        mian += p.fitness 
        if p.fitness == 1024:
          return p 
 
      print('mian', float(mian / len(popu)))
      popu = self.kill_the_pop(popu)
      
      if popu == None:
          return None 

      print('length', len(popu))
     

      

      popu = self.create_new_pop(popu)
        


In [22]:
ft = Fitness('truth_table.csv', 10)



gen_al = Genetic(ft, 1000, 0.05, 0.7, 0.2)

sol = gen_al.run_gen()
if sol != None:
  print('found the best', sol.get_names())
else:
  print('not the best', gen_al.best.get_names())




mian 667.715
992
length 800
mian 701.785268414482
992
length 641
mian 725.2886115444618
1004
length 513
mian 746.715953307393
1004
length 412
mian 767.9854721549636
1004
length 331
found the best ['and', 'xor', 'or', 'xnor', 'and', 'or', 'nand', 'xnor', 'or']


In [30]:
ft2 = Fitness('truth_table.csv', 10)

print(ft2.check_fitness(sol))
print(ft2.check_real_fitness(sol))


1024
0


From the above results one can conclud that we foud the negative of the solution Therefore the real solution is:
['and', 'xor', 'or', 'xnor', 'and', 'or', 'nand', 'xnor', 'nor']

In [31]:
genes = [And_gate(), Xor_gate(), Or_gate(), Xnor_gate(), And_gate(), Or_gate(), Nand_gate(), Xnor_gate(), Nor_gate()]
final_chr = Chromosom(genes)
print(ft2.check_real_fitness(final_chr))

1024
